In [1]:
import pandas as pd 
import numpy as np 
import spacy

from spacy.tokens import Doc, DocBin
from spacy.matcher import PhraseMatcher
from spacy import displacy

from tqdm.notebook import tqdm
from collections import Counter, defaultdict
import pickle
from pathlib import Path
import random
from joblib import Parallel, delayed

from collections import Counter
import phonetics 


from itertools import islice
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

Doc.set_extension("pdf_path", default=None)


spacy.__version__

'3.1.1'

In [2]:
# The documents saved in a previous notebook 

#!  python -m spacy download en_core_web_sm
model = "en_core_web_lg"

text_path  = Path('texts.pkl')
docs_path  = Path(f'docs-{model}.spacy')
nlp_path = Path(f'nlp-{model}.spacy')

nlp = spacy.load(model)
nlp.max_length = 3000000

nlp = nlp.from_disk(nlp_path)
doc_bin = DocBin().from_disk(docs_path)
docs = list(doc_bin.get_docs(nlp.vocab))

In [3]:
# Map of types of entities. 
import re
p = re.compile(r'\W+')

docv = list(docs)

wd = defaultdict(list)

for d in docv:
    for e in  d.ents:
        if not str(e).isnumeric():
            wd[e.label_].append(p.sub(' ', str(e.lemma_)).lower() )

cd = defaultdict(Counter)

for k, v in wd.items():
    cd[k].update(v)


In [4]:
%%time
# Make a single combined document
from spacy.tokens import Doc
cdocs = Doc.from_docs(docv)

<timed exec>:3: UserWarning: [W101] Skipping Doc custom extension 'pdf_path' while merging docs.


CPU times: user 48.7 s, sys: 10.8 s, total: 59.5 s
Wall time: 1min 4s


In [5]:
# Match to words that indicate referencing an out reach program
import spacy
from spacy.matcher import PhraseMatcher
from itertools import islice
from tqdm.notebook import tqdm

phrase_matcher = PhraseMatcher(nlp.vocab)
phrases = ['outreach']
patterns = [nlp(text) for text in phrases]
phrase_matcher.add('outreach', None, *patterns)

contr_shall = {}

i = 0

contr_matches = {}
for doc in tqdm(list(docs)):
  
    for match_id, start, end in phrase_matcher(doc):
        span = doc[start:end]
        contr_matches[(id(doc), start, end)] = span.sent

len(contr_matches)
      

  0%|          | 0/876 [00:00<?, ?it/s]

3500

In [25]:
agrmt = pd.read_csv('agreement_statements.csv').iloc[:,1:]

or_docs =  pd.DataFrame([ {'path':v.doc._.pdf_path, 'or_sent': str(v)} for k,v in contr_matches.items()])

or_docs = or_docs.merge(agrmt)
or_docs = or_docs[['or_sent','contractor']].drop_duplicates().dropna()
print(len(or_docs))
or_docs.head()

2270


,or_sent,contractor
0,Additional NAMI SD services include monthly\nE...,Nami
1,"We do this by offering free classes,\nsupport ...",Nami
2,Location ofservices and outreach.,Nami
3,NAMI SD's outreach includes our staff working ...,Nami
5,Contractor's Short Term Case Manager(STCM)for ...,Mcalister Institute


In [26]:
or_docs.to_csv('outreach.csv')


In [29]:
or_docs.sort_values('contractor')['contractor'].unique()

array(['Alpha Project', 'Casa De Amparo', 'Catholic Charities',
       'Center For Community Solutions',
       'Community Health Improvement Partners',
       'Community Research Foundation', 'Community Resource Center',
       'Corporation For Supportive Housing',
       'Deaf Community Services Of San Diego',
       'Episcopal Community Services', 'Exodus Recovery',
       'Family Health Centers Of San Diego', 'Healthright', 'Home Start',
       'Interfaith', 'Legal Aid Society Of San', 'Mcalister Institute',
       'Mental Health Systems', 'Nami', 'National Alliance',
       'Neighborhood House Association', 'New Alternatives',
       'North County Lifeline', 'Pan Asian Communities', 'Path',
       'People Assisting', 'Prosperity Way', 'Public Consulting Group',
       'Recovery Innovations', 'Regional Task Force On The Homeless',
       'Rescare Workforce Services', 'San Diego Youth Services',
       'Serving Seniors', 'South Bay Community Services',
       'Southern Indian Health